<a href="https://colab.research.google.com/github/jorgemarquesferreirajunior/YOLO/blob/main/ContagemDeObjetosDetectados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detectando objetos com YOLO e OpenCV - Explorando mais opções

## Etapa 1 - Importando as bibliotecas

In [ ]:
import cv2
print(cv2.__version__)

In [ ]:
!pip install opencv-python==4.4.0.40

In [ ]:
import cv2
import numpy as np
import time
import os
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import zipfile
print(cv2.__version__)

## Etapa 2 - Conectando com o Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## Etapa 3 - Carregando os arquivos do modelo treinado

In [ ]:
path = '/content/gdrive/My Drive/Cursos - recursos/YOLO/modelo_YOLOv4.zip'
zip_object = zipfile.ZipFile(file=path, mode="r")
zip_object.extractall("./")
zip_object.close()

In [ ]:
labelsPath = os.path.sep.join(['/content/cfg', "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

In [ ]:
weightsPath = os.path.sep.join(['/content/', "yolov4.weights"])
configPath = os.path.sep.join(['/content/cfg', "yolov4.cfg"])

In [ ]:
net = cv2.dnn.readNet(configPath, weightsPath)

## Etapa 4 - Definindo mais configurações para a detecção

In [ ]:
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

In [ ]:
ln = net.getLayerNames()
print("Todas as camadas (layers):")
print(ln)
print("Total: "+ str(len(ln)))
print("Camadas de saída: ")
print(net.getUnconnectedOutLayers())
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print(ln)

## Etapa 5 - Carregando a imagem onde será feita a detecção

In [ ]:
def mostrar(img):
  fig = plt.gcf()
  fig.set_size_inches(16, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
imagem = cv2.imread("/content/gdrive/My Drive/Cursos - recursos/YOLO/imagens/cachorros02.jpg")
mostrar(imagem)
imagem_cp = imagem.copy()
(H, W) = imagem.shape[:2]
print("Altura: " + str(H) + "\nLargura: " + str(W))

## Etapa 6 - Processando a imagem de entrada

### Redimensionamento da imagem (opcional)

In [ ]:
imagem.shape

In [ ]:
proporcao = imagem.shape[1] / imagem.shape[0]
proporcao

In [ ]:
1500 / 1.5

In [ ]:
def redimensionar(imagem, largura_maxima = 600):
  if imagem.shape[1] > largura_maxima:
    proporcao = imagem.shape[1] / imagem.shape[0]
    imagem_largura = largura_maxima
    imagem_altura = int(imagem_largura / proporcao)
  else:
    imagem_largura = imagem.shape[1]
    imagem_altura = imagem.shape[0]

  imagem = cv2.resize(imagem, (imagem_largura, imagem_altura))
  return imagem

In [ ]:
imagem = redimensionar(imagem)

In [ ]:
mostrar(imagem)
(H, W) = imagem.shape[:2]
imagem_cp = imagem.copy()
print('Altura: ' + str(H) + '\nLargura: ' + str(W))

### Construindo o blob da imagem

In [ ]:
def blob_imagem(net, imagem, mostrar_texto=True):
  inicio = time.time()

  blob = cv2.dnn.blobFromImage(imagem, 1 / 255.0, (416, 416), swapRB=True, crop=False)
  net.setInput(blob)
  layerOutputs = net.forward(ln)

  termino = time.time()

  if mostrar_texto:
    print("YOLO levou {:.2f} segundos".format(termino - inicio))

  return net, imagem, layerOutputs

In [ ]:
net, imagem, layerOutputs = blob_imagem(net, imagem)

## Etapa 7 - Definindo as variáveis

In [ ]:
_threshold = 0.5
_threshold_NMS = 0.3
caixas = []
confiancas = []
IDclasses = []

## Etapa 8 - Realizando a predição

In [ ]:
def deteccoes(detection, _threshold, caixas, confiancas, IDclasses):
  scores = detection[5:]
  classeID = np.argmax(scores)
  confianca = scores[classeID]

  if confianca > _threshold:
      caixa = detection[0:4] * np.array([W, H, W, H])
      (centerX, centerY, width, height) = caixa.astype("int")
      x = int(centerX - (width / 2))
      y = int(centerY - (height / 2))

      caixas.append([x, y, int(width), int(height)])
      confiancas.append(float(confianca))
      IDclasses.append(classeID)

  return caixas, confiancas, IDclasses

In [ ]:
for output in layerOutputs:
    for detection in output:
        caixas, confiancas, IDclasses = deteccoes(detection, _threshold, caixas, confiancas, IDclasses)

In [ ]:
print(caixas)
print(confiancas)
print(IDclasses)

In [ ]:
len(caixas)

## Etapa 9 - Aplicando a Non-Maxima Suppression

In [ ]:
objs = cv2.dnn.NMSBoxes(caixas, confiancas, _threshold, _threshold_NMS)

In [ ]:
print("Objetos detectados: " + str(len(objs)))

## Etapa 10 - Mostrando o resultado da detecção na imagem

In [ ]:
def check_negativo(n):
  if (n < 0):
    return 0
  else:
    return n

def funcoes_imagem(imagem, i, confiancas, caixas, COLORS, LABELS, mostrar_texto=True):
  (x, y) = (caixas[i][0], caixas[i][1])
  (w, h) = (caixas[i][2], caixas[i][3])
  cor = [int(c) for c in COLORS[IDclasses[i]]]
  cv2.rectangle(imagem, (x, y), (x + w, y + h), cor, 2)
  texto = "{}: {:.4f}".format(LABELS[IDclasses[i]], confiancas[i])
  if mostrar_texto:
    print("> " + texto)
    print(x,y,w,h)
  cv2.putText(imagem, texto, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, cor, 2)

  return imagem,x,y,w,h

In [ ]:
if len(objs) > 0:
  for i in objs.flatten():
    imagem, x, y, w, h = funcoes_imagem(imagem, i, confiancas, caixas, COLORS, LABELS)
    objeto = imagem_cp[y:y + h, x:x + w]
    cv2_imshow(objeto)

In [ ]:
mostrar(imagem)

# Fazendo a detecção em múltiplas imagens de uma vez

In [ ]:
path = '/content/gdrive/My Drive/Cursos - recursos/YOLO/imagens/fotos_teste.zip'
zip_object = zipfile.ZipFile(file = path, mode = 'r')
zip_object.extractall('./')
zip_object.close()

In [ ]:
diretorio_fotos = 'fotos_teste'
caminhos = [os.path.join(diretorio_fotos, f) for f in os.listdir(diretorio_fotos)]
print(caminhos)

In [ ]:
for caminho_imagem in caminhos:
  try:
    imagem = cv2.imread(caminho_imagem)
    (H, W) = imagem.shape[:2]
  except:
    print('Erro ao carregar a imagem -> ' + caminho_imagem)
    continue

  imagem_cp = imagem.copy()
  net, imagem, layer_outputs = blob_imagem(net, imagem)

  caixas = []
  confiancas = []
  IDclasses = []

  for output in layer_outputs:
    for detection in output:
      caixas, confiancas, IDclasses = deteccoes(detection, _threshold, caixas, confiancas, IDclasses)

  objs = cv2.dnn.NMSBoxes(caixas, confiancas, _threshold, _threshold_NMS)

  if len(objs) > 0:
    for i in objs.flatten():
      imagem, x, y, w, h = funcoes_imagem(imagem, i, confiancas, caixas, COLORS, LABELS, mostrar_texto=False)
      objeto = imagem_cp[y:y + h, x:x + w]

  mostrar(imagem)

# Contando quantas vezes algum objeto específico apareceu em múltiplas imagens

In [ ]:
diretorio_fotos = "fotos_teste/"
caminhos = [os.path.join(diretorio_fotos, f) for f in os.listdir(diretorio_fotos)]
print(caminhos)
threshold = 0.5
threshold_NMS = 0.3

In [ ]:
contagem = 0
contagem_total = 0

for caminho_imagem in caminhos:
  try:
    imagem = cv2.imread(caminho_imagem)
    (H, W) = imagem.shape[:2]
  except:
    print('Erro ao carregar a imagem -> ' + caminho_imagem)
    continue

  contagem = 0

  imagem_cp = imagem.copy()
  net, imagem, layer_outputs = blob_imagem(net, imagem)

  caixas = []
  confiancas = []
  IDclasses = []

  for output in layer_outputs:
    for detection in output:
      caixas, confiancas, IDclasses = deteccoes(detection, threshold, caixas, confiancas, IDclasses)

  objs = cv2.dnn.NMSBoxes(caixas, confiancas, threshold, threshold_NMS)

  if len(objs) > 0:
    for i in objs.flatten():

      if LABELS[IDclasses[i]] == 'person':
        contagem += 1
        contagem_total += 1

      imagem, x, y, w, h = funcoes_imagem(imagem, i, confiancas, caixas, COLORS, LABELS, mostrar_texto=False)
      objeto = imagem_cp[y:y + h, x:x + w]

  print('Pessoas detectadas na imagem ' + str(caminho_imagem) + ': ' + str(contagem))
  mostrar(imagem)

In [ ]:
print('Total de pessoas detectadas: ' + str(contagem_total))

# Fazendo a detecção apenas de objetos específicos do modelo

In [ ]:
diretorio_fotos = "fotos_teste/"
caminhos = [os.path.join(diretorio_fotos, f) for f in os.listdir(diretorio_fotos)]
print(caminhos)
threshold = 0.5
threshold_NMS = 0.3

In [ ]:
classes = ['dog', 'cat']
for caminho_imagem in caminhos:
  try:
    imagem = cv2.imread(caminho_imagem)
    (H, W) = imagem.shape[:2]
  except:
    print('Erro ao carregar a imagem -> ' + caminho_imagem)
    continue

  imagem_cp = imagem.copy()
  net, imagem, layer_outputs = blob_imagem(net, imagem)

  caixas = []
  confiancas = []
  IDclasses = []

  for output in layer_outputs:
    for detection in output:
      caixas, confiancas, IDclasses = deteccoes(detection, _threshold, caixas, confiancas, IDclasses)

  objs = cv2.dnn.NMSBoxes(caixas, confiancas, _threshold, _threshold_NMS)

  if len(objs) > 0:
    for i in objs.flatten():
      if LABELS[IDclasses[i]] in classes:
        imagem, x, y, w, h = funcoes_imagem(imagem, i, confiancas, caixas, COLORS, LABELS, mostrar_texto=False)
        objeto = imagem_cp[y:y + h, x:x + w]

  mostrar(imagem)

# Modo mais legível

In [ ]:
def funcoes_imagem_v2(imagem, i, confiancas, caixas, COLORS, LABELS, mostrar_texto=True):
  (x, y) = (caixas[i][0], caixas[i][1]) # coordenada (x,y) onde inicia a caixa da detecção
  (w, h) = (caixas[i][2], caixas[i][3]) # largura e altura em pixels da caixa de detecção

  cor = [int(c) for c in COLORS[IDclasses[i]]]

  fundo = np.full((imagem.shape), (0,0,0), dtype=np.uint8)

  texto = "{}: {:.4f}".format(LABELS[IDclasses[i]], confiancas[i])

  cv2.putText(fundo, texto, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)

  fx,fy,fw,fh = cv2.boundingRect(fundo[:,:,2])

  cv2.rectangle(imagem, (x, y), (x + w, y + h), cor, 2)

  cv2.rectangle(imagem, (fx, fy), (fx + fw, fy + fh), cor, -1)
  cv2.rectangle(imagem, (fx, fy), (fx + fw, fy + fh), cor, 3)
  cv2.putText(imagem, texto, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)

  if mostrar_texto:
    print("> " + texto)
    print(x,y,w,h)

  return imagem,x,y,w,h

In [ ]:
diretorio_fotos = "fotos_teste/"
caminhos = [os.path.join(diretorio_fotos, f) for f in os.listdir(diretorio_fotos)]
print(caminhos)
threshold = 0.5
threshold_NMS = 0.3

In [ ]:
for caminho_imagem in caminhos:
  try:
    imagem = cv2.imread(caminho_imagem)
    (H, W) = imagem.shape[:2]
  except:
    print('Erro ao carregar a imagem -> ' + caminho_imagem)
    continue

  imagem_cp = imagem.copy()
  net, imagem, layer_outputs = blob_imagem(net, imagem)

  caixas = []
  confiancas = []
  IDclasses = []

  for output in layer_outputs:
    for detection in output:
      caixas, confiancas, IDclasses = deteccoes(detection, _threshold, caixas, confiancas, IDclasses)

  objs = cv2.dnn.NMSBoxes(caixas, confiancas, _threshold, _threshold_NMS)

  if len(objs) > 0:
    for i in objs.flatten():
      imagem, x, y, w, h = funcoes_imagem_v2(imagem, i, confiancas, caixas, COLORS, LABELS, mostrar_texto=False)
      objeto = imagem_cp[y:y + h, x:x + w]

  mostrar(imagem)